# Stable Diffusion WebUI (Automatic1111) for SageMaker Studio Lab

You will need a ngrok auth token to run it, as gradio doesn't work on SageMaker Studio Lab

If you don't have a ngrok account yet, create one here: https://dashboard.ngrok.com/signup

It's free for non-commercial use

# Installation: Clone webui repository
***You only need to do this once on your SageMaker SL account***

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# Installation: Download model(s) from huggingface
***You only need to do this once on your SageMaker SL account***

Alternatively you can use the JupyterLab file browser to upload one or more .ckpt files to the ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/ folder

### Download model(s)


#### Check free disk space

In [ ]:
!df -h | grep -E 'Avail|home'

#### Huggingface token

In [ ]:
import ipywidgets as widgets

token_textbox = widgets.Text(
    value='Enter Huggingface token here',
    description='Token:',
)
token_textbox

The next cell will download model ***v1-5-pruned-emaonly.ckpt*** (4GB) 

This model can be used for training textual inversion embeddings and hypernetworks

Comment out and uncomment lines in the next cell to download different models

Go to these pages and ***accept the licenses*** before running the next cell:

https://huggingface.co/runwayml/stable-diffusion-v1-5

https://huggingface.co/CompVis/stable-diffusion-v-1-4-original

https://huggingface.co/runwayml/stable-diffusion-inpainting

In [ ]:
user_header = f"\"Authorization: Bearer {token_textbox.value}\""
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/

# Model 1.4
#!wget --header={user_header} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

# Model 1.5 (8GB) - only necessary for native training, not for embeddings/hypernetworks
#!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt 

# Model 1.5 (4GB)
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt 

# Inpainting model 1.5 (4GB)
#!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt

### Optional: Download variational autoencoder (VAE)

Doing so can improve the quality of generated images, if you select the VAE in the webui settings

Go to these pages and ***accept the licenses*** before running the next cell:

https://huggingface.co/stabilityai/sd-vae-ft-ema-original

https://huggingface.co/stabilityai/sd-vae-ft-mse-original

In [ ]:
# sd-vae-ft-ema-original
#%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/VAE/
#!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt

# sd-vae-ft-mse-original
#%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/VAE/
#!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt

# sd-vae-ft-mse-original
# Use this if you downloaded v1-5-pruned-emaonly.ckpt above and want to autoselect the VAE every time you load this model in the webui
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/
!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O v1-5-pruned-emaonly.vae.pt 



###  Optional: Download waifu division model and VAE

From https://huggingface.co/hakurei/waifu-diffusion-v1-4

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/
!wget --header={user_header} https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/models/wd-1-3-penultimate-ucg-cont.ckpt

%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/VAE/
!wget --header={user_header} https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt
!wget --header={user_header} https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt

### Optional: Download a model from Google Drive

Copy the file ID from your url and paste it into the GOOGLE_FILE_ID variable below, before running this cell

In [ ]:
%pip install gdown

In [ ]:
###########################################################
GOOGLE_FILE_ID = "1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA"
###########################################################

# above file ID is Stable Diffusion model 1.4. Replace it with the file ID of the model you want to download from Google Drive

import gdown

%cd /content/stable-diffusion-webui/models/Stable-diffusion/
url = "https://drive.google.com/u/0/uc?id=" + GOOGLE_FILE_ID + "&export=download&confirm=t"
print("Downloading " + url)
gdown.download(url)


### Optional: Download a model from MEGA

In [ ]:
Change MEGA_DOWNLOAD_URL to the URL of the model, then run the cell

Download can be very slow (>30 minutes for a 4GB model)

In [ ]:
##########################################################
MEGA_DOWNLOAD_URL = "https://mega.nz/file/mdcVARhL#FUq5TL2xp7FuzzgMS4B20sOYYnPZsyPMw93sPMHeQ78"
##########################################################

%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/models/Stable-diffusion/

%pip install mega.py
from mega import Mega

mega = Mega()
print("Connecting to MEGA")
m = mega.login()
print("Downloading file (no progress display available)")
m.download_url(MEGA_DOWNLOAD_URL)
print("Download complete")

# Installation: Uninstall opencv-python to prevent libgthread error when starting webui

***You only need to do this once on your SageMaker SL account***

This section is a unnecessary mess, but it works. It may be enough to just run the !conda install part. Did not test it yet, as I'd have to create a new account for that

Not running this section will lead to libgthread-2.0.so.0 errors when starting the webui

In [ ]:
%pip uninstall opencv-python -y
%pip install opencv-python-headless

In [ ]:
# this will restart the kernel
exit()

In [ ]:
%pip uninstall opencv-python-headless -y

In [ ]:
!conda install glib=2.51.0 -y

# Update: Change into Web UI directory and download updates
This is not strictly necessary. You can run this every time before you start the webui

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui
!git pull

## Update: Install webui requirements.txt
This is necessary sometimes after you did a !git pull above

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui
%pip install -r requirements.txt

# Run: Enter ngrok auth token


In [ ]:
import ipywidgets as widgets

ngrok_token_textbox = widgets.Text(
    value='Enter ngrok auth token here',
    description='Token:',
)
ngrok_token_textbox

# Run: Launch web ui

You will get a ngrok.io link. Follow it and enter the username and password shown below. You may want to change username and password for security reasons.

In [ ]:
##################################

WEBUI_USERNAME="genius"
WEBUI_PASSWORD="artist"

##################################

ARGS = "\"--gradio-debug --gradio-auth " + WEBUI_USERNAME + ":" + WEBUI_PASSWORD + " --ngrok " + ngrok_token_textbox.value + "\""
#ARGS = "\"--gradio-debug --gradio-auth " + WEBUI_USERNAME + ":" + WEBUI_PASSWORD + " --share\""

%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui
!COMMANDLINE_ARGS=$ARGS REQS_FILE="requirements.txt" python launch.py


Commandline arguments are:
  - `--share` - create online gradio.app link (currently this doesn't work on SageMaker SL)
  - `--gradio-debug` - print outputs to console
  - `--ngrok` - start ngrok reverse proxy with supplied auth token

End: Commands for ***after*** you have gotten done with a session
============================================================================
Click the square stop button before running these cells

### Create .tar.gz archive of images for downloading 

Location will be ***~/sagemaker-studiolab-notebooks/outputs.tar.gz***

In [ ]:
%cd ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/
!echo "Creating archive. Please wait..."
!tar -czf ~/sagemaker-studiolab-notebooks/outputs.tar.gz outputs
!echo "You can now delete the generated images and download the archive"

### Delete generated images from outputs folder

In [ ]:
!rm -rf ~/sagemaker-studiolab-notebooks/stable-diffusion-webui/outputs/*

### Restart kernel and delete logfile (this kills webui and ngrok)

In [ ]:
!rm ~/sagemaker-studiolab-notebooks/ngrok/ngrok.log
exit()